In [119]:
import json
import requests
from bs4 import BeautifulSoup

import os

In [120]:
def docx_html_url(_id): 
    return f'https://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id={_id}'

In [121]:
def articles_url(_number, _violation): 
    violation = 'nonviolation'
    if(_violation): violation = violation[3:]
        
    return f'https://hudoc.echr.coe.int/app/query/results?query=contentsitename%3AECHR%20AND%20(NOT%20(doctype%3DPR%20OR%20doctype%3DHFCOMOLD%20OR%20doctype%3DHECOMOLD))%20AND%20((languageisocode%3D%22ENG%22))%20AND%20((documentcollectionid%3D%22GRANDCHAMBER%22)%20OR%20(documentcollectionid%3D%22CHAMBER%22))%20AND%20(({violation}%3D{_number}))&select=sharepointid,Rank,ECHRRanking,languagenumber,itemid,docname,doctype,application,appno,conclusion,importance,originatingbody,typedescription,kpdate,kpdateAsText,documentcollectionid,documentcollectionid2,languageisocode,extractedappno,isplaceholder,doctypebranch,respondent,ecli,appnoparts,sclappnos&sort=&start=20&length=100&rankingModelId=11111111-0000-0000-0000-000000000000'

In [122]:
def articles_json(_number, _violation):
    results = []
    violation = 'nv'
    if(_violation): violation = violation[1:] 
    
    for result in json_data['results']:
        results.append({'id': result["columns"]["itemid"], 'name': result["columns"]["docname"], 'violation': violation})
    
    return results

In [123]:
def article_get(_id):
    url = docx_html_url(_id)
    
    r = requests.get(url).text
    soup = BeautifulSoup(r, "html5lib")
    
    for script in soup(["script", "style"]):
        script.extract()
        
    text = soup.get_text()
    
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = text.replace('\xa0', ' ')
    
    return text

In [124]:
def articles_generator(_number, _violation):
    violation = 'nonviolation'
    if _violation: violation = violation[3:]
        
    path = f'hudoc/Article{_number}/{violation}'
    
    os.makedirs(path, exist_ok=True)
    
    for article in articles_json(_number, _violation):
        f = open(f'{path}/{article["id"]}.txt', 'w')
        f.write(article_get(article["id"]))

In [118]:
from multiprocessing.dummy import Pool as ThreadPool 

articles = [
    ['3', True],
    ['3', False],
    ['6', True],
    ['6', False],
    ['8', True],
    ['8', False]
]
    
pool = ThreadPool(6) 
pool.starmap(articles_generator, zip(articles_num, articles_vio))

/home/nti14147326/.local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 884 of the file /usr/lib/python3.6/threading.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


KeyboardInterrupt: 